### Credit Assesment Classification Project

#### Imports

In [38]:
import sklearn
import pandas as pd
import numpy as np
from dateutil import relativedelta

In [39]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

C:\Users\mateu\AppData\Local\Temp\ipykernel_14832\3212007839.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train=pd.read_csv("train.csv")


#### Getting Baseline for Classification

In [40]:
baseline_acc_train=train["Credit_Score"].value_counts()
baseline_acc_train

Credit_Score
Standard    53174
Poor        28998
Good        17828
Name: count, dtype: int64

In [41]:
total_samples = baseline_acc_train.sum()
majority_count = baseline_acc_train.max()
percentage_majority = (majority_count / total_samples) * 100

print("Baseline prediction when guessing majority class for training dataset: {:.2f}%".format(percentage_majority))

Baseline prediction when guessing majority class for training dataset: 53.17%


#### Familiarizing with the data and Clean-up

##### Fixing the occurences of _ in float/int type objects

Getting derived column from Social Security Number

In [42]:
train['SSN_Location'] = train['SSN'].str[:3]

Converting values to floats/integers and removing '_' signs (errors in data)

In [43]:
for column in ['Annual_Income',"Age", 'Num_of_Loan', 'Num_of_Delayed_Payment', "Changed_Credit_Limit", "Outstanding_Debt", "Amount_invested_monthly","Monthly_Balance","SSN_Location"]:
    train[column] = train[column].str.replace('_', '')

for column in ['Age', 'Num_of_Loan', "SSN_Location"]:
    try:
        train[[column]] = train[[column]].astype(int)
    except:
        pass

for column in train[['Annual_Income', 'Num_of_Delayed_Payment', "Changed_Credit_Limit", "Outstanding_Debt", "Amount_invested_monthly","Monthly_Balance"]]:
    try:
        train[[column]] = train[[column]].astype(float)
    except:
        pass

##### Dealing with some NA values

In [ ]:
train['Monthly_Inhand_Salary'] = train.groupby('Customer_ID')['Monthly_Inhand_Salary'].transform(lambda x: x.fillna(x.median()))
train['Monthly_Inhand_Salary'].fillna(train['Monthly_Inhand_Salary'].median(), inplace=True)

In [ ]:
train.sort_values(['Customer_ID', 'Month'], inplace=True)
train['Num_of_Delayed_Payment'].fillna(method='ffill', inplace=True)
train['Num_of_Delayed_Payment'].fillna(train['Num_of_Delayed_Payment'].median(), inplace=True)

In [ ]:
train['Num_Credit_Inquiries'].fillna(method='ffill', inplace=True)
train['Num_Credit_Inquiries'].fillna(train['Num_Credit_Inquiries'].median(), inplace=True)

In [ ]:
train['Type_of_Loan'].fillna('Missing_Information', inplace=True)

In [ ]:
train['Amount_invested_monthly'] = train.groupby('Customer_ID')['Amount_invested_monthly'].transform(lambda x: x.fillna(x.median()))
train["Amount_invested_monthly"].fillna(train['Amount_invested_monthly'].median(), inplace=True)

In [ ]:
train['Monthly_Balance'] = train.groupby('Customer_ID')['Monthly_Balance'].transform(
    lambda x: x.fillna(x.median() if x.notna().any() else np.nan)
)

train["Monthly_Balance"].fillna(train['Monthly_Balance'].median(), inplace=True)


In [ ]:
train.sort_values(['Customer_ID', 'Month', 'Credit_History_Age'], inplace=True)
train['Credit_History_Age'] = train.groupby('Customer_ID')['Credit_History_Age'].transform(
    lambda x: x.fillna(x.ffill().bfill())
)

def calculate_month_difference(start_month, end_month):
    start_year = 1
    end_year = 1

    if start_month > end_month:
        end_year = 2

    months_diff = abs(start_month - end_month)
    years_diff = end_year - start_year

    return relativedelta.relativedelta(months=months_diff, years=years_diff)

for index, row in train.iterrows():
    if pd.isna(row['Credit_History_Age']):
        client_id = row['Customer_ID']
        month_num = row['Month']
        
        prev_row = train.loc[(train['Customer_ID'] == client_id) & (train['Month'] < month_num), 'Credit_History_Age'].dropna().tail(1)
        next_row = train.loc[(train['Customer_ID'] == client_id) & (train['Month'] > month_num), 'Credit_History_Age'].dropna().head(1)
        
        if not prev_row.empty and not next_row.empty:
            prev_age = pd.to_timedelta(prev_row.values[0])
            next_age = pd.to_timedelta(next_row.values[0])
            
            month_diff = calculate_month_difference(prev_age.months, next_age.months)
            true_age = prev_age + month_diff
            
            train.at[index, 'Credit_History_Age'] = str(true_age.years) + ' Years and ' + str(true_age.months) + ' Months'



train['Credit_History_Age'].fillna(train['Credit_History_Age'].mode()[0], inplace=True)

##### Fixing invalid values and dealing with anomalies

Fixing anomalies for 'Age' column

In [ ]:
min_age = 10
max_age = 150

train.sort_values(by=['Customer_ID', 'Month'], inplace=True)
train.reset_index(drop=True, inplace=True)

for i in range(1, len(train)):
    if train.loc[i, 'Age'] < min_age or train.loc[i, 'Age'] > max_age:
        if train.loc[i, 'Customer_ID'] == train.loc[i-1, 'Customer_ID']:
            train.loc[i, 'Age'] = train.loc[i-1, 'Age']

median_age = train['Age'].median()
for i in range(1, len(train)):
    if train.loc[i, 'Age'] < min_age or train.loc[i, 'Age'] > max_age:
        train.loc[i, 'Age'] = median_age

Fixing anomalies for 'Annual_Income' column

In [ ]:
min_income = 0
max_income = 10
num_changed_values=0

train.sort_values(by=['Customer_ID', 'Month'], inplace=True)
train.reset_index(drop=True, inplace=True)

for i in range(1, len(train)):
    wrong_income=False
    if train.loc[i, 'Customer_ID'] == train.loc[i-1, 'Customer_ID'] and (train.loc[i, 'Annual_Income']*max_income<train.loc[i-1, 'Annual_Income'] or train.loc[i, 'Annual_Income']>max_income*train.loc[i-1, 'Annual_Income']):
        wrong_income=True
    if train.loc[i, 'Annual_Income'] < min_income or wrong_income:
        try:
            if train.loc[i, 'Customer_ID'] == train.loc[i-1, 'Customer_ID'] or train.loc[i, 'Customer_ID'] == train.loc[i+1, 'Customer_ID']:
                if train.loc[i, 'Annual_Income'] < min_income:
                    train.loc[i, 'Annual_Income']=max(train.loc[i-1, 'Annual_Income'],train.loc[i, 'Annual_Income'])
                else:
                    train.loc[i, 'Annual_Income'] = min(train.loc[i-1, 'Annual_Income'],train.loc[i, 'Annual_Income'])
                num_changed_values+=1
        except:
            pass
print(num_changed_values)

986


Fixing anomalies for 'Num_Bank_Accounts' column

In [ ]:
min_accounts = 0
max_accounts = 100
num_changed_values=0

train.sort_values(by=['Customer_ID', 'Month'], inplace=True)
train.reset_index(drop=True, inplace=True)

for i in range(0, len(train)-1):
    if train.loc[i, 'Num_Bank_Accounts'] < min_accounts or train.loc[i, 'Num_Bank_Accounts'] > max_accounts:
        if train.loc[i, 'Customer_ID'] == train.loc[i+1, 'Customer_ID'] and train.loc[i, 'Num_Bank_Accounts'] != train.loc[i+1, 'Num_Bank_Accounts']:
            train.loc[i, 'Num_Bank_Accounts'] = train.loc[i+1, 'Num_Bank_Accounts']
            num_changed_values+=1
print(num_changed_values)

1103


Fixing anomalies for 'Num_Credit_Card' column

In [ ]:
min_cards = 0
max_cards = 100
num_changed_values=0

train.sort_values(by=['Customer_ID', 'Month'], inplace=True)
train.reset_index(drop=True, inplace=True)

for i in range(0, len(train)-1):
    if train.loc[i, 'Num_Credit_Card'] < min_cards or train.loc[i, 'Num_Credit_Card'] > max_cards:
        if train.loc[i, 'Customer_ID'] == train.loc[i+1, 'Customer_ID'] and train.loc[i, 'Num_Credit_Card'] != train.loc[i+1, 'Num_Credit_Card']:
            train.loc[i, 'Num_Credit_Card'] = train.loc[i+1, 'Num_Credit_Card']
            num_changed_values+=1
print(num_changed_values)

1858


Fixing anomalies for 'Interest_Rate' column

In [ ]:
upper_range=np.percentile(train['Interest_Rate'], 98)
lower_range=0

median_value = train['Interest_Rate'].median()
train.loc[(train['Interest_Rate'] > upper_range) | (train['Interest_Rate'] < lower_range), 'Interest_Rate'] = median_value

Fixing anomalies for 'Num_of_Loan' column

In [ ]:
upper_range=np.percentile(train['Num_of_Loan'], 99.5)
lower_range=0

median_value = train['Num_of_Loan'].median()
train.loc[(train['Num_of_Loan'] > upper_range) | (train['Num_of_Loan'] < lower_range), 'Num_of_Loan'] = median_value

Fixing anomalies for 'Type_of_Loan' column

In [ ]:
unique_values = set()
for value in train['Type_of_Loan']:
    for sub_value in value.split(', '):
        unique_values.add(sub_value.replace("and ",""))

for value in unique_values:
    train[value] = train['Type_of_Loan'].apply(lambda x: 1 if value in x else 0)

train.drop(columns=["Type_of_Loan"],inplace=True)

Fixing anomalies for 'Num_of_Delayed_Payment' column

In [ ]:
min_delays = 0
max_delays = 5
num_changed_values=0

train.sort_values(by=['Customer_ID', 'Month'], inplace=True)
train.reset_index(drop=True, inplace=True)

for i in range(1, len(train)):
    wrong_income=False
    if train.loc[i, 'Customer_ID'] == train.loc[i-1, 'Customer_ID'] and (train.loc[i, 'Num_of_Delayed_Payment']*max_income<train.loc[i-1, 'Num_of_Delayed_Payment'] or train.loc[i, 'Num_of_Delayed_Payment']>max_income*train.loc[i-1, 'Num_of_Delayed_Payment']):
        wrong_income=True
    if train.loc[i, 'Num_of_Delayed_Payment'] < min_income or wrong_income:
        try:
            if train.loc[i, 'Customer_ID'] == train.loc[i-1, 'Customer_ID'] or train.loc[i, 'Customer_ID'] == train.loc[i+1, 'Customer_ID']:
                if train.loc[i, 'Num_of_Delayed_Payment'] < min_income:
                    train.loc[i, 'Num_of_Delayed_Payment']=max(train.loc[i-1, 'Num_of_Delayed_Payment'],train.loc[i, 'Num_of_Delayed_Payment'])
                else:
                    train.loc[i, 'Num_of_Delayed_Payment'] = min(train.loc[i-1, 'Num_of_Delayed_Payment'],train.loc[i, 'Num_of_Delayed_Payment'])
                num_changed_values+=1
        except:
            pass
print(num_changed_values)

3197


In [ ]:
upper_range=np.percentile(train['Num_of_Delayed_Payment'], 99.8)
lower_range=0

median_value = train['Num_of_Delayed_Payment'].median()
train.loc[(train['Num_of_Delayed_Payment'] > upper_range) | (train['Num_of_Delayed_Payment'] < lower_range), 'Num_of_Delayed_Payment'] = median_value

Fixing anomalies for 'Changed_Credit_Limit' column

In [ ]:
num_changed_values=0
train.sort_values(by=['Customer_ID', 'Month'], inplace=True)
train.reset_index(drop=True, inplace=True)

for i in range(0, len(train)-1):
    if train.loc[i, 'Changed_Credit_Limit']=='':
        if train.loc[i, 'Customer_ID'] == train.loc[i+1, 'Customer_ID']:
            train.loc[i, 'Changed_Credit_Limit'] = train.loc[i+1, 'Changed_Credit_Limit']
        else:
            train.loc[i, 'Changed_Credit_Limit'] = 0
        num_changed_values+=1

for i in range(0,len(train)):
    if train.loc[i, 'Changed_Credit_Limit']=='':
        train.loc[i, 'Changed_Credit_Limit'] = 0
        num_changed_values+=1

train[['Changed_Credit_Limit']] = train[['Changed_Credit_Limit']].astype(float)
print(num_changed_values)

2123


Fixing anomalies for 'Num_Credit_Inquiries' column

In [ ]:
upper_range=np.percentile(train['Num_Credit_Inquiries'], 98.3)
lower_range=0

train.sort_values(by=['Customer_ID', 'Month'], inplace=True)
train.reset_index(drop=True, inplace=True)

for i in range(0, len(train)-1):
    if train.loc[i, 'Num_Credit_Inquiries'] < lower_range or train.loc[i, 'Num_Credit_Inquiries'] > upper_range:
        if train.loc[i, 'Customer_ID'] == train.loc[i+1, 'Customer_ID'] and train.loc[i, 'Num_Credit_Inquiries'] != train.loc[i+1, 'Num_Credit_Inquiries']:
            train.loc[i, 'Num_Credit_Inquiries'] = train.loc[i+1, 'Num_Credit_Inquiries']
            num_changed_values+=1
        try:
            if train.loc[i, 'Customer_ID'] == train.loc[i-1, 'Customer_ID'] and train.loc[i, 'Num_Credit_Inquiries'] != train.loc[i-1, 'Num_Credit_Inquiries']:
                train.loc[i, 'Num_Credit_Inquiries'] = train.loc[i-1, 'Num_Credit_Inquiries']
                num_changed_values+=1
        except:
            pass

median_value = train['Num_Credit_Inquiries'].median()
for i in range(0,len(train)-1):
    if train.loc[i, 'Num_Credit_Inquiries'] < lower_range or train.loc[i, 'Num_Credit_Inquiries'] > upper_range:
        train.loc[i,"Num_Credit_Inquiries"] = median_value

print(num_changed_values)

4195


Adding derived data from 'Credit_History_Age' column and dropping the original column

In [ ]:
train[['Years', 'Months']] = train['Credit_History_Age'].str.extract(r'(\d+) Years and (\d+) Months')
train[['Years', 'Months']] = train[['Years', 'Months']].astype(int)
train.drop(columns="Credit_History_Age",inplace=True)

Changing values for 'Occupation' to be more human-understandable, assuming ______ in 'Occupation' means 'unemployed'

In [ ]:
to_replace = '_______'
replacement = 'Unemployed'

train['Occupation'] = train['Occupation'].replace(to_replace, replacement)

Changing values for 'Credit_Mix' to be more human-understandable, assuming '_' in 'Credit_Mix' means 'Missing'

In [ ]:
to_replace = '_'
replacement = 'Missing'

train['Credit_Mix'] = train['Credit_Mix'].replace(to_replace, replacement)

Changing values for 'Payment_Behaviour' to be more human-understandable, assuming '!@9#%8' in 'Payment_Behaviour' means 'Missing'

In [ ]:
to_replace = '!@9#%8'
replacement = 'Missing'

train['Payment_Behaviour'] = train['Payment_Behaviour'].replace(to_replace, replacement)

##### Dropping unwanted columns and adding derived columns

Dropping ID, Customer_ID and Name columns. Extracting first 3 values from SSN column, adding column to show the result of classification for the previous loan request made by the client.

In [ ]:
train = train.sort_values(["Customer_ID", 'Month'])
train['Previous_Classification'] = ''
for index, row in train.iterrows():
    if index > 0 and train.at[index-1, "Customer_ID"] == row["Customer_ID"]:
        train.at[index, 'Previous_Classification'] = train.at[index-1, 'Credit_Score']
    else:
        train.at[index, 'Previous_Classification'] = "NCR"

train.drop(columns=["ID","Customer_ID", "Name","SSN"],inplace=True)

#### Saving the cleaned-up dataframe

##### Current state of the dataframe

In [ ]:
train

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,...,Missing_Information,Home Equity Loan,Debt Consolidation Loan,Credit-Builder Loan,Auto Loan,Payday Loan,Mortgage Loan,Years,Months,Previous_Classification
0,April,17,Lawyer,30625.94,2706.161667,6,5,27,2,64,...,0,1,0,1,0,0,0,10,5,NCR
1,August,18,Lawyer,30625.94,2706.161667,6,5,27,2,57,...,0,1,0,1,0,0,0,10,9,Poor
2,February,17,Lawyer,30625.94,2706.161667,6,5,27,2,62,...,0,1,0,1,0,0,0,10,3,Poor
3,January,17,Lawyer,30625.94,2706.161667,6,5,27,2,62,...,0,1,0,1,0,0,0,10,2,Poor
4,July,18,Lawyer,30625.94,2706.161667,6,5,27,2,62,...,0,1,0,1,0,0,0,10,8,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,January,29,Scientist,41398.44,3749.870000,8,7,13,6,25,...,0,1,0,0,1,1,1,18,2,Standard
99996,July,30,Scientist,41398.44,3749.870000,8,7,13,6,23,...,0,1,0,0,1,1,1,18,8,Standard
99997,June,29,Scientist,41398.44,3749.870000,8,7,13,6,23,...,0,1,0,0,1,1,1,18,7,Good
99998,March,29,Scientist,41398.44,3749.870000,8,7,13,6,25,...,0,1,0,0,1,1,1,18,4,Standard


##### Saving the datframe to CSV

In [ ]:
train.to_csv("train_clean", index=False)